In [1]:
# Install svn package to download folder
package = ! dpkg -s subversion | grep Status
if "Status: install ok installed" not in package:
  !sudo apt-get install subversion
# Download dataset folders from github
# Install svn package to download folder
package = ! dpkg -s subversion | grep Status
if "Status: install ok installed" not in package:
  !sudo apt-get install subversion
# Download dataset folders from github
! svn checkout https://github.com/tote93/Big-Data-SVR/trunk/dataset/

Password:
   C dataset/original
   A dataset/original/CO2 march.xls
   A dataset/original/co2 april.xls
   A dataset/original/humidity april.xls
   A dataset/original/humidity march.xls
   A dataset/original/temperature1 april.xls
   A dataset/original/temperature1 march.xls
   A dataset/original/temperature2 april.xls
   A dataset/original/temperature2 march.xls
Revisión obtenida: 3


# 1. Import the Libraries

In [2]:
#To handle data in the form of rows and columns
import pandas as pd
#For numerical libraries
import numpy as np
# Import Path for move files
from pathlib import Path
# Avoid show warning msg
import warnings
# Measure time and generate files libraries
import os, time
# Check if file has a word in filename
import fnmatch 
# Sklearn libraries for ml
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# Sklearn libraries for testint models
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
# Sklearn standarScale libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
# Support Vector Regressor library
from sklearn.svm import SVR
# Graphic and plot libraries
from matplotlib import pyplot
#importing ploting libraries
import matplotlib.pyplot as plt
#To enable plotting graphs in Jupyter notebook
%matplotlib inline
#importing seaborn for statistical plots
import seaborn as sns
# Plot and seaborn configuration
plt.rc('font',size=14)
sns.set(style='white')
sns.set(style='whitegrid',color_codes=True)
warnings.filterwarnings('ignore')
# Add posibles missing values to search in the dataset
missing_values = ["n/a", "na", "--", "?", "NaN"]

In [3]:
# Processing all dataset files

In [4]:
def datetimeClean(data):
    dti = pd.to_datetime(data['Date'])
    data['Date'] = dti.dt.round("T")
    # Replace all seconds to 0, to start all in the same time
    data['Date'] = data['Date'].apply(lambda t: t.replace(second=0))
    data.drop_duplicates(subset ="Date", keep = "first", inplace = True)
    return data

In [5]:
# Read data from file
for file in os.listdir("./dataset/original"):
    if fnmatch.fnmatch(file, '*'):
        # create  dataframe from excel
        data = pd.read_excel("./dataset/original/"+file)
        # Clean datatime, round to next minute
        data = datetimeClean(data)
        # Create the final dataframe with the original columns
        df_final = pd.DataFrame(columns = data.columns)
        # Reformating Datetime 
        end_month = pd.to_datetime(data['Date'][len(data)-1]).month
        df = data
        # Reformate datetime value
        df['Date'] = df['Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
        # Transform date from string to datetime type
        df['Date'] = pd.to_datetime(df['Date'])
        # set the time column as index
        df = df.set_index(['Date'])
        # target: date_range 1 min steps
        tgt = pd.date_range(df.index[0], df.index[-1], freq='T')
        # now use .isin() and negate to get the timestamps that are missing in df
        missing_data = tgt[~tgt.isin(df.index)]
        # Append all missing data
        for date in missing_data:
            data = data.append(pd.Series([date, np.nan], index=data.columns), ignore_index=True)

        # Parse data to datetime, could be more efficient using a dictionary, loop O(N)
        data['Date'] = pd.to_datetime(data['Date'])
        for i, _date in enumerate(data['Date']):      
            # Ignore dates out the initial range
            if pd.to_datetime(_date).month == end_month:                  
                df_final = df_final.append(pd.Series([_date, round(data['Y_Value'][i],2)], index=data.columns), ignore_index=True)
        # Sort values and export
        df_final = df_final.sort_values(by=['Date'])
        # Export to excel file
        df_final.to_excel(file, index=False)
        print("exported:",file, "NumPatrones:",len(df_final))        
print("Finished exporting files")

exported: temperature1 april.xls NumPatrones: 35418
exported: humidity april.xls NumPatrones: 35418
exported: CO2 march.xls NumPatrones: 38412
exported: humidity march.xls NumPatrones: 38411
exported: co2 april.xls NumPatrones: 35418
exported: temperature1 march.xls NumPatrones: 38410
exported: temperature2 march.xls NumPatrones: 38411
exported: temperature2 april.xls NumPatrones: 35418
Finished exporting files


# Compute all new files to merge into a month file

In [6]:
os.makedirs('file_output')
os.makedirs('dataset/final')
# Open all files from march
df1 = pd.read_excel("temperature1 march.xls")
df2 = pd.read_excel("temperature2 march.xls")
df3 = pd.read_excel("humidity march.xls")
df4 = pd.read_excel("CO2 march.xls")
# Rename column and add new colums from other datasets
df1 = df1.rename(columns={"Y_Value": "Temp1"})
df1['Temp2'] = df2['Y_Value']
df1['Humi'] = df3['Y_Value']
df1['CO2'] = df4['Y_Value']
# Finally export to a new dataset
df1.to_csv("file_output/march_output.csv", index=False, header=True, decimal=".", sep=",", float_format='%.2f')

# Open all files from april
df1 = pd.read_excel("temperature1 april.xls")
df2 = pd.read_excel("temperature2 april.xls")
df3 = pd.read_excel("humidity april.xls")
df4 = pd.read_excel("co2 april.xls")
# Rename column and add the other attirbutes
df1 = df1.rename(columns={"Y_Value": "Temp1"})
df1['Temp2'] = df2['Y_Value']
df1['Humi'] = df3['Y_Value']
df1['CO2'] = df4['Y_Value']
# Export to a new dataset
df1.to_csv("file_output/april_output.csv", index=False, header=True, decimal=".", sep=",", float_format='%.2f')


In [7]:
# Clean all secondary files
dir_name = "."
test = os.listdir(dir_name)
for item in test:
    if item.endswith(".xls"):
        os.remove(os.path.join(dir_name, item))

# Load the dataset

In [8]:
#reading the CSV file into pandas dataframe
data = pd.read_csv("file_output/april_output.csv", na_values = missing_values, sep=",", parse_dates=True)

In [9]:
#Check top few records of the dataset
data.head()

,Date,Temp1,Temp2,Humi,CO2
0,2019-04-01 02:00:00,24.66,23.3,27.84,424.96
1,2019-04-01 02:01:00,24.64,23.3,27.84,424.96
2,2019-04-01 02:02:00,24.66,23.3,27.84,422.72
3,2019-04-01 02:03:00,24.64,23.3,27.84,421.76
4,2019-04-01 02:04:00,24.64,23.3,27.84,419.84


*   It shows that there are four independent variables ( Date, Temp1, Temp2, Humi) and one dependent variable (CO2).
*   All the records are numeric but Date attribute.

In [10]:
#Check the last few records of the dataset
data.tail()

,Date,Temp1,Temp2,Humi,CO2
35413,2019-04-25 16:13:00,27.64,26.9,34.9,406.72
35414,2019-04-25 16:14:00,27.62,26.9,34.9,405.76
35415,2019-04-25 16:15:00,27.62,26.9,34.9,406.72
35416,2019-04-25 16:16:00,27.62,26.9,34.9,408.00
35417,2019-04-25 16:17:00,27.62,26.9,34.9,404.80


# 3. Exploratory data quality report April Dataset


>## 3.2 Multivariate analysis

> ## 3.3. Strategies to handle different data challenges

#### Checking for Missing Values

**Handling the outliers**

#### Preprocessing datasets


#### Output Dataset Visualization

# SVR Parameters Optimization

## **Parameters to optimice**
**C (Regularisation)**: C is the penalty parameter. The higher the Penalty value the model makes fewer mistakes but it is observed that after a certain Penalty value the model is over training.

**Gamma**: It defines how far influences the calculation of plausible line of separation. The higher the gamma value, only near points are considered.
A lower gamma value, far away points are also considered.

**Degree**: Higher degree polynomial kernels allow a more flexible decision boundary.

## Optimizing Parameters


### Generate Algorithms Comparison


### Testing with the full dataset preprocesed

### Divide dataset in train - test files and preprocess individually


### Now we Fit the model of previous preprocesed
